In [9]:
# === IMPORTAZIONE LIBRERIRE ===
import cv2
import os
import re
import pandas as pd
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes
import itertools
import scipy.io as sio

In [ ]:
# === Function to deproject pixel to 3d point (without distortion) ===

def deproject_pixel_to_point(intr, x, y, depth):
    X = (x - intr['ppx']) / intr['fx'] * depth
    Y = (y - intr['ppy']) / intr['fy'] * depth
    Z = depth
    return [X, Y, Z]

# === Function to compute 3d euclidean distance ===

def euclidean_distance_3d(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

# === Save surface data to .mat

def save_surface_to_mat(name, raw_emotion, X, Y, Z, output_folder=None):
    if output_folder is None:
        output_folder = r'C:\Users\Tatiana Cordoba\OneDrive - Politecnico di Torino\Lab SG3D condiviso\progetto\surfaceZ_racccolti2'

    os.makedirs(output_folder, exist_ok=True)
    filename = f"{name}_{raw_emotion}.mat"
    output_path = os.path.join(output_folder, filename)

    data_dict = {'X': X, 'Y': Y, 'Z': Z}
    sio.savemat(output_path, data_dict)
    return output_path

# ===  RILEVAMENTO LANDMARKS FACCIALI ===

def calculate_all_distances_from_image(rgb_path, raw_path, name, raw_emotion, landmark_ids=None):

    # === Load RGB image ===
    rgb_image = cv2.imread(rgb_path)
    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)

    # === Load Depth image (.RAW) ===
    depth_raw = np.fromfile(raw_path, dtype=np.uint16).reshape((480, 640))

    # === Initialization FaceMesh ===
    mp_face_mesh = mp.solutions.face_mesh 
    face_mesh = mp_face_mesh.FaceMesh( 
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True 
        )
    results = face_mesh.process(rgb_image) 

    if not results.multi_face_landmarks:
        print(f"No landmarks detected in {rgb_path}")
        return []
     
    # === Fill holes in the Depth image ===
    binary_mask = depth_raw > 0
    filled_mask = binary_fill_holes(binary_mask)
    holes_mask = filled_mask.astype(np.uint8) - binary_mask.astype(np.uint8) 

    # === MANUAL FILLING OF DETECTED HOLES (average of valid pixels 3x3 neighbors) ===

    depth_filled = depth_raw.copy()
    h, w = depth_filled.shape
    for i in range(1, h-1):
        for j in range(1, w-1):
            if holes_mask[i, j]:
                neighbors = depth_filled[i-1:i+2, j-1:j+2]
                valid_neighbors = neighbors[neighbors > 0]
                if valid_neighbors.size > 0:
                    depth_filled[i, j] = np.mean(valid_neighbors)
    
    # === FACE SEGMENTATION ===

    mask = np.zeros((rgb_image.shape[0], rgb_image.shape[1]), dtype=np.uint8)
    all_points = []

    # === Denormalizes landmark coordinates ===

    if results.multi_face_landmarks:
        for lm in results.multi_face_landmarks[0].landmark:
            x = int(lm.x * rgb_image.shape[1]) 
            y = int(lm.y * rgb_image.shape[0]) 
            all_points.append((x, y))

        
        hull = cv2.convexHull(np.array(all_points, dtype=np.int32))
        cv2.fillConvexPoly(mask, hull, 255)
        mask = cv2.dilate(mask, np.ones((3, 3), np.uint8))  # expansión

    # Apply the face mask to the Depth map
    depth_face = cv2.bitwise_and(depth_filled, depth_filled, mask=mask)
    
    # Removes invalid depth values (0 and >7000).
    depth_face = depth_face.astype(np.float32)
    depth_face[depth_face == 0] = np.nan 
    depth_face[depth_face > 7000] = np.nan 
    Z = -depth_face 
    X, Y = np.meshgrid(np.arange(Z.shape[1]), np.arange(Z.shape[0]))

    # === Save surface data to .mat ===
    save_surface_to_mat(name, raw_emotion, X, Y, Z)

    # === Landmark IDs ===
    if landmark_ids is None:
        landmark_ids = [70, 105, 55, 285, 334, 300, 33, 133, 362, 263, 
                        48, 4, 278, 61, 0, 291, 13, 14, 17, 199, 168, 122, 351]
        
    id_to_acronym = {
        70: 'oebsx', 105: 'mebsx', 55: 'iebsx', 285: 'iebdx', 334: 'mebdx', 300: 'oebdx',
        33: 'exsx', 133: 'ensx', 362: 'endx', 263: 'exdx', 48: 'alsx', 4: 'prn',
        278: 'aldx', 61: 'chsx', 0: 'ls', 291: 'chdx', 13: 'Sto', 14: 'sl',
        17: 'li',199: 'Men', 168: 'se', 122: 'nossx', 351: 'nosdx'
    }

    # === Selected distances ===
    distance_labels = {'dist_euc_2':'oebsx-iebsx',
    'dist_euc_3': 'oebsx-iebdx',
    'dist_euc_5': 'oebsx-oebdx',
    'dist_euc_6': 'oebsx-exsx',
    'dist_euc_7': 'oebsx-ensx',
    'dist_euc_12': 'oebsx-alsx',
    'dist_euc_13': 'oebsx-prn',
    'dist_euc_44': 'iebsx-iebdx',
    'dist_euc_47': 'iebsx-exsx',
    'dist_euc_48': 'iebsx-ensx',
    'dist_euc_49': 'iebsx-endx',
    'dist_euc_50': 'iebsx-exdx',
    'dist_euc_53': 'iebsx-alsx',
    'dist_euc_54': 'iebsx-prn',
    'dist_euc_55': 'iebsx-aldx',
    'dist_euc_63': 'iebsx-se',
    'dist_euc_65': 'iebdx-oebdx',
    'dist_euc_66': 'iebdx-exsx',
    'dist_euc_67': 'iebdx-ensx',
    'dist_euc_68': 'iebdx-endx',
    'dist_euc_69': 'iebdx-exdx',
    'dist_euc_72': 'iebdx-alsx',
    'dist_euc_73': 'iebdx-prn',
    'dist_euc_74': 'iebdx-aldx',
    'dist_euc_82': 'iebdx-se',
    'dist_euc_103': 'oebdx-endx',
    'dist_euc_104': 'oebdx-exdx',
    'dist_euc_108': 'oebdx-prn',
    'dist_euc_109': 'oebdx-aldx',
    'dist_euc_120': 'exsx-exdx',
    'dist_euc_123': 'exsx-alsx',
    'dist_euc_124': 'exsx-prn',
    'dist_euc_125': 'exsx-aldx',
    'dist_euc_133': 'exsx-se',
    'dist_euc_134': 'ensx-endx',
    'dist_euc_138': 'ensx-alsx',
    'dist_euc_139': 'ensx-prn',
    'dist_euc_140': 'ensx-aldx',
    'dist_euc_148': 'ensx-se',
    'dist_euc_149': 'endx-exdx',
    'dist_euc_152': 'endx-alsx',
    'dist_euc_153': 'endx-prn',
    'dist_euc_154': 'endx-aldx',
    'dist_euc_162': 'endx-se',
    'dist_euc_165': 'exdx-alsx',
    'dist_euc_166': 'exdx-prn',
    'dist_euc_167': 'exdx-aldx',
    'dist_euc_175': 'exdx-se',
    'dist_euc_200': 'alsx-aldx',
    'dist_euc_208': 'alsx-se',
    'dist_euc_217': 'prn-se',
    'dist_euc_225': 'aldx-se'}
    
    # === INTRINSIC PARAMETERS AND DEPTH SCALE ===

    intrinsics = {
        'fx': 474.346,
        'fy': 474.346,
        'ppx': 309.568,
        'ppy': 245.154,
        'distortion': [0.139663, 0.0914142, 0.00468509, 0.00220023, 0.0654529],
        'model': 'Inverse Brown Conrady'
    }
    depth_scale = 0.000125  # To meters

    # ===  Extract 3D points ===

    landmark_dict = {}
    for idx in landmark_ids:
        lm = results.multi_face_landmarks[0].landmark[idx]
        x = int(lm.x * rgb_image.shape[1])
        y = int(lm.y * rgb_image.shape[0])
        if 0 <= x < Z.shape[1] and 0 <= y < Z.shape[0]:
            depth = Z[y, x] #* depth_scale
            if not np.isnan(depth):
                landmark_dict[idx] = (x , y, depth)


    # ===  Calculate all pairwise 3D distances ===

    distances = {}
    landmark_ids_present = sorted(landmark_dict.keys())

    for id1, id2 in itertools.combinations(landmark_ids_present, 2):

        x1, y1, z1 = landmark_dict[id1]
        x2, y2, z2 = landmark_dict[id2]

        d1 = z1 * depth_scale
        d2 = z2 * depth_scale

        p1 = deproject_pixel_to_point(intrinsics, x1, y1, d1)
        p2 = deproject_pixel_to_point(intrinsics, x2, y2, d2)

        acronym1 = id_to_acronym[id1]
        acronym2 = id_to_acronym[id2]
        pair_to_find = '-'.join(sorted([acronym1, acronym2]))

        dist_name = None
        for key, value in distance_labels.items():
            value_pair = '-'.join(sorted(value.split('-')))
            if value_pair == pair_to_find:
                dist_name = key
                break
        if dist_name is None:
            dist_name = f'{acronym1}-{acronym2}'

        dist = euclidean_distance_3d(p1, p2)
        distances[dist_name] = dist*100

    return distances


def process_all_images(root_path):
    import os
    import pandas as pd
    import re

    all_data = []

    for person_name in os.listdir(root_path):
        person_folder = os.path.join(root_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        # List all PNG and RAW files
        png_files = [f for f in os.listdir(person_folder) if f.endswith('.png')]
        raw_files = [f for f in os.listdir(person_folder) if f.endswith('.raw')]

        # Helper to extract name and raw emotion from filename
        def get_prefix_parts(filename):
            parts = filename.split('_')
            if len(parts) < 2:
                return None, None
            return parts[0], parts[1]  # name, raw_emotion

        for png_file in png_files:
            name, raw_emotion = get_prefix_parts(png_file)
            if not name or not raw_emotion:
                continue

            # Find a matching RAW file with the same name and emotion prefix
            raw_match = next(
                (raw for raw in raw_files if get_prefix_parts(raw)[0] == name and get_prefix_parts(raw)[1] == raw_emotion),
                None
            )
            if raw_match is None:
                print(f"WARNING: No RAW file found for {png_file}")
                continue

            # Clean the emotion by removing any trailing digits (e.g., 'serio1' → 'serio')
            emotion = re.sub(r'\d+$', '', raw_emotion)

            png_path = os.path.join(person_folder, png_file)
            raw_path = os.path.join(person_folder, raw_match)

            # Calculate distances (your custom function)
            distances = calculate_all_distances_from_image(
                png_path, raw_path, name,raw_emotion, landmark_ids=None
            )

            # Store results in a dictionary
            row = {
                'name': name,
                'emotion': emotion
            }
            row.update(distances)
            all_data.append(row)

    # Build and return DataFrame
    df = pd.DataFrame(all_data)
    return df

def process_and_export_distances(df):
    # Filter distance columns
    distance_columns = [col for col in df.columns if col.startswith('dist_euc_')]

    # Sort columns numerically
    distance_columns_sorted = sorted(distance_columns, key=lambda x: int(x.split('_')[-1]))

    # Reorganize the DataFrame
    ordered_columns = ['name', 'emotion'] + distance_columns_sorted
    df = df[ordered_columns]

    # Round distances to 1 decimal
    df.loc[:, distance_columns_sorted] = df[distance_columns_sorted].round(1)

    # Convert DataFrame to dictionary for MATLAB (optional)
    matlab_dict = {}
    matlab_dict['name'] = df['name'].values.astype(str)
    matlab_dict['emotion'] = df['emotion'].values.astype(str)
    for col in distance_columns_sorted:
        matlab_dict[col] = df[col].values

    # Save DataFrame to .csv
    df.to_csv('Data.csv', index=False)
    
    return df, matlab_dict



In [23]:
root_path = r'C:\Users\Tatiana Cordoba\OneDrive - Politecnico di Torino\Lab SG3D condiviso\progetto\acquisizioni_realsense'
df_final = process_all_images(root_path)
df_processed, data_dict = process_and_export_distances(df_final)

In [24]:
df_processed

,name,emotion,dist_euc_2,dist_euc_3,dist_euc_5,dist_euc_6,dist_euc_7,dist_euc_12,dist_euc_13,dist_euc_44,...,dist_euc_154,dist_euc_162,dist_euc_165,dist_euc_166,dist_euc_167,dist_euc_175,dist_euc_200,dist_euc_208,dist_euc_217,dist_euc_225
0,antonio,felice,4.8,7.1,11.4,1.9,4.4,7.2,8.8,2.7,...,3.9,2.1,8.3,7.2,5.1,5.0,4.2,4.7,4.5,4.7
1,antonio,serio,5.4,7.7,11.7,2.3,4.9,7.9,9.4,2.6,...,4.2,2.1,8.2,7.4,5.5,5.2,3.8,4.7,4.5,4.8
2,antonio,serio,4.2,6.8,11.5,1.9,4.0,7.0,8.3,2.7,...,4.1,2.0,8.3,7.4,5.4,5.1,3.9,4.8,4.6,4.8
3,antonio,sorriso,4.5,7.1,11.6,1.8,4.3,7.3,8.8,2.8,...,4.2,2.0,8.3,7.4,5.4,5.0,4.1,4.7,4.6,4.8
4,antonio,sorriso,4.8,7.2,11.6,2.0,4.4,7.6,9.1,2.8,...,4.2,2.0,8.3,7.3,5.3,5.0,4.1,4.8,4.6,4.9
5,elena,felice,4.0,6.3,NaN,1.6,3.9,5.8,7.7,2.4,...,3.9,2.3,7.5,7.1,5.2,5.0,4.0,4.2,4.1,4.2
6,elena,seria,4.2,6.5,11.5,1.7,3.9,6.4,8.1,2.5,...,4.1,2.2,7.5,7.4,5.5,4.9,3.4,4.3,4.3,4.3
7,elena,seria,4.0,6.5,11.4,1.7,3.9,6.5,8.0,2.5,...,4.2,2.3,7.7,7.3,5.7,5.0,3.2,4.3,4.3,4.4
8,elena,sorriso,4.0,6.3,11.2,1.6,3.8,6.0,7.9,2.4,...,4.0,2.3,7.5,7.3,5.5,5.0,3.5,4.3,4.3,4.2
9,elena,sorriso,4.3,6.5,11.1,1.7,4.0,6.6,8.3,2.4,...,3.8,2.2,7.5,7.2,5.1,4.8,3.4,4.3,4.2,4.2
